| 现象               | 说明                             | 对模型的影响                         |
|--------------------|----------------------------------|--------------------------------------|
| 新测试集集中分布   | 存在分布偏移（Domain shift）     | 模型可能不再有效，需重评泛化能力     |
| 与训练集部分重叠   | 仍有少量语义交集                 | 有迁移可能，但需要调整建模策略       |
| PCA 清晰地区分     | 嵌入向量确实含有语义结构信息     | 可继续使用嵌入特征进行深度建模       |


你观察得很到位 ✅  
确实，项目已经提供了固定的训练集和两个测试集，那你的问题是：

> **既然官方已经划分好训练集了，我们还需要自己做 stratified sampling 吗？**

---

## ✅ 简短答案：

> **是的，在训练集内部划分验证集时，仍然建议使用 Stratified Sampling。**

---

## 🧠 原因解释如下：

### 🔹 项目给的是：  
| 文件名 | 用途 | 是否有标签 | 用来干嘛 |
|--------|------|------------|----------|
| `X_train.csv`, `y_train.csv` | 训练数据 | ✅ | 模型训练（你来决定怎么验证） |
| `X_test_1.csv` | 测试集 1 | ❌ | 最终评估（用于提交） |
| `X_test_2.csv`, `y_test_2_reduced.csv` | 新分布数据 | ✅（仅前202行） | 用于最后任务，检测模型是否泛化 |

---

### ✅ 为什么还需要划分训练集 → 训练集 + 验证集？

在训练阶段：

- 我们需要用部分训练数据来“调模型”、“做 early stopping”、“评估泛化能力”
- 如果我们**随机划分**训练/验证集，在极度不平衡数据中，验证集中可能缺少少数类
- 所以我们必须使用：

```python
from sklearn.model_selection import train_test_split

X_train_part, X_val, y_train_part, y_val = train_test_split(
    X_train, y_train, stratify=y_train, test_size=0.2, random_state=42
)
```

这样才能保证 **验证集中保留和训练集一样的类别分布结构**，从而避免验证结果失真。

---

### ✅ 示例图演示效果（你可能已经看到）

假如不使用 `stratify=y`，验证集可能会：

- 几乎没有类别 1、2、3
- 或者某些类别在验证集中只出现 1 个样本，导致评估指标不稳定

---

## ✅ 总结一段话（可用于 EDA/报告）：

```markdown
虽然项目已提供固定的训练集和测试集划分，但我们在训练阶段仍需要从训练集中进一步划分出验证集。由于数据存在严重类别不平衡，我们采用 stratified sampling 方式进行训练/验证集的划分，以确保验证集能保留与原训练集一致的类别分布结构，从而避免模型评估结果偏差。
```

在开始构建分类模型之前，我们必须先了解训练集中标签的分布情况。

因为：

- 如果数据集是均衡的，模型就有机会公平地学习每一个类别；

- 但在现实中，数据往往是高度不平衡的，某些类别样本特别多，某些类别样本极少。

这一步的结果将直接影响我们后续的策略选择，比如：

- 是否需要加权训练（class_weight="balanced"）；

- 是否需要对少数类进行上采样（如 SMOTE）；

- 是否应该采用宏平均的 F1-score，而不是单纯的 accuracy。